# Automated Statistical Adjustment of Volumetric MRI Data

In this Jupyter notebook there are functions and explanations as to how to use each tool in the Head Size Correction package. Functions can be customized to fit individual research needs.

This Jupyter notebook also serves as a tutorial tool for head size correction. This is intended to standardize methods used in volumetric estimation of subregions in longitudinal MRI data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns

Start with importing numpy, matplotlib.pyplot, pandas, and various other packages to Python. Functions in these packages are embedded into the functions for the correction and for outlier detection. 

In [2]:
def estimate_coef(x,y):
    n = np.size(x)
    m_x, m_y = np.mean(x), np.mean(y)
    
    SS_xy = np.sum(y*x) - n * m_y * m_x
    SS_xx = np.sum(x*x) - n * m_x * m_x
    
    b_1 = SS_xy / SS_xx
    b_0 = m_y - b_1 * m_x
    
    return b_0,b_1

This function returns the coefficients of linear regression. This coefficient will be embedded into various correction functions, regression functions, and residual functions. However, it may not be necessary to compute this value in your analysis, due to it being embedded in other functions. 

You can also use np.poly1d function to do the same thing, however this function is integrated in the rest of the functions. 

## Correction Methods 

The next two functions are various correction methods used for volumetric subregions. According to O'Brien et.al. 2011, there are 3 types of correction methods: proportional methods, ANCOVA methods, and residual methods. In this version of the software we use 2 types of methods, proportional and ANCOVA. There are two types of proportional methods used, one that is strictly the proportion of the ROI to the ICV, and one that is described in Liu 2014 that uses a power proportion method. 

In [3]:
def ancova_volume(ICV, region):
    mean_ICV = np.mean(ICV)
    b = estimate_coef(ICV, region)
    B = b[1]
    adjusted_volume = region - B*(ICV - mean_ICV)
    return adjusted_volume

This function corrects a region based on headsize for ICV. It is according to the **insert citation here** method. *Insert Pros and Cons of this method?*

In [4]:
def proportion(ICV, region):
    
    proportion = region/ICV
    
    adjusted_volume = np.mean(ICV)*proportion
    
    return adjusted_volume

In [5]:
def power_proportion(ICV,region):
    from scipy.optimize import curve_fit
    def curve(x,a,b):
        return a*x**b
    
    parameters, whatever = curve_fit(curve,ICV,region)
    
    a = parameters[0]
    b = parameters[1]
    
    proportion = region/(ICV**b)
    
    adjusted_volume = np.mean(ICV**b) * proportion
    
    return adjusted_volume


This function corrects a region based on ICV based on the power-proportion method described in Liu 2014. The original method, the proportion method ... *pros and cons of this method*

## Residual, Leverage, and Outlier Detection

The next group of functions are in place to account for extreme values in the data. Best practice of science accounts for data that may cause too much influence in the analysis to be representative of the sample. These functions help to streamline the process of finding and dealing with outliers. 

In [6]:
def coef_of_determination(x, y):
    
    b = estimate_coef(x,y)
    
    SS_tot_array = np.array([])
    SS_res_array = np.array([])
    
    y_pred = b[0] + b[1] * x
    for i in range(len(x)):
        SS_tot_array = np.append(SS_tot_array, (y[i] - np.mean(y))**2)
        SS_res_array = np.append(SS_res_array, (y[i] - y_pred[i])**2)
    
    SS_tot = np.sum(SS_tot_array)
    SS_res = np.sum(SS_res_array)
    
    r_squared = 1 - (SS_res/SS_tot)
        
    return r_squared
    
def residual_list(x,y):
    
    b = estimate_coef(x,y)
    
    SS_tot_array = np.array([])
    SS_res_array = np.array([])
    
    y_pred = b[0] + b[1] * x
    for i in range(len(x)):
        SS_tot_array = np.append(SS_tot_array, (y[i] - np.mean(y))**2)
        SS_res_array = np.append(SS_res_array, (y[i] - y_pred[i])**2)
    
    SS_tot = np.sum(SS_tot_array)
    SS_res = np.sum(SS_res_array)
        
    return SS_res_array

In [7]:
def leverage(region, ICV, method = "residual/ancova"):

    if method == "proportion":
        volume_adjusted = proportion(ICV, region)
        
    elif method == "power proportion":
        volume_adjusted = power_proportion(ICV,region)
    
    else:
        volume_adjusted = adjust_volume(region, region)
        
    h = np.array([])
    for i in range(len(volume_adjusted)):
        hi = (region[i] - volume_adjusted[i])/volume_adjusted[i]
        h = np.append(h, hi)    
        
    return h

def check_leverage(region: np.array, leverage:np.array) -> np.array:
    n = len(region)
    outliers = np.array([])
    for i in leverage:
        if i > 4/n:
            outliers = np.append(outliers, i)
            
    return outliers

The coef_of_determination function returns the coefficient of determination. This is useful in determining how well the data fits a linear curve. Once head size is corrected for, this value should be close to 0 as variation due to head size will have been taken out of the data.

The residual_list function returns a numpy array of residuals in the data. 

The leverage function returns a numpy array of leverages in the data.

The check_leverage function returns a numpy array of extreme leverages in the data. This uses a threshold of 1/(4n) as a high value.

## High Leverage and Influential Points

In [8]:
def calculate_leverage(icv):
    ICV = np.array(icv)
    ones = np.ones_like(ICV)
    
    data = np.vstack((ones, ICV))
    data_matrix_T = np.matrix(data)
    X = data_matrix_T.T
    
    H = X * (X.T * X) ** (-1) * X.T
    x = np.diag(H)
    h = 2/len(x)
    
    high_leverage = np.array([])
    for i in x:
        if i > (3 * h):
            high_leverage = np.append(high_leverage, i)
    
    locations = np.array([])
    for i in range(len(high_leverage)):
        j = int(i)
        location = np.where(x == high_leverage[j])
        locations = np.append(locations, location)
        
    return high_leverage, locations


In [9]:
# HOW TO DO COOK'S DISTANCE FOR INFLUENTIAL POINTS
'''
m = smf.ols(formula = "ICV ~ Cbl",data = df).fit()
infl = m.get_influence()
sm_fr = infl.summary_frame()
cooks_distance = sm_fr["cooks_d"]
'''

def high_cooks_values(cooks_distance):
    outliers = np.array([])
    locations = np.array([])
    for i in cooks_distance:
        if i > 4/len(cooks_distance): # this can be changed to some other threshold 
            outliers = np.append(outliers, i)
            
    for i in range(len(outliers)):
        j = int(i)
        location = np.where(cooks_distance == outliers[j])
        locations = np.append(locations, location)
        
    return outliers, locations


In [10]:
def plot_regression_line(x, y):
    b = estimate_coef(x,y)
    plt.scatter(x, y, color = "b", marker = "o", s=30)
    
    y_pred = b[0] + b[1] * x
    
    plt.plot(x,y_pred, color = "g")
    
    plt.xlabel('x')
    plt.ylabel('y')
    
    plt.show

## Filtering by Categorical Data

In [11]:
def filter_by_sex(sex, region):
    female_region = np.array([])
    male_region = np.array([])
    for i in range(len(sex)):
        if sex[i] == 'f':
            female_region = np.append(female_region, region[i])
        else:
            male_region = np.append(male_region, region[i])
            
    return(female_region, male_region)

In [12]:
def hbp_filter(hbp, region):
    yes_region = np.array([])
    no_region = np.array([])
    for i in range(len(sex)):
        if hbp[i] == 0:
            no_region = np.append(no_region, region[i])
        else:
            yes_region = np.append(yes_region, region[i])
            
    return(yes_region, no_region)

In [13]:
df = pd.read_excel(r'C:\Users\kayle\Dropbox\PA 2019-2020\Raz_Study10_Raw-Manual-Volumes_for_ICV_adjust.xlsx')

In [14]:
def correction(data, region_s, ICV_s, method = "ancova", leverage_check = True):
    region = data[region_s]
    ICV = data[ICV_s]
    
    
    ## leverage check
    if leverage_check == True:
        
        formula_string = ICV_s +"~"+ region_s
        
        m = smf.ols(formula = formula_string, data = df).fit()
        infl = m.get_influence()
        sm_fr = infl.summary_frame()
        cooks_distance = sm_fr["cooks_d"]
        
        cooks_outliers = high_cooks_values(cooks_distance)
        if len(cooks_outliers[0]) == 0:
            print(cooks_outliers[1])
            for j in cooks_outliers[1]:
                data = data.drop()
        else: 
            print(cooks_outliers)
            
        
    ## correction step
    if method == "ancova":
         volume_adjusted = ancova_volume(ICV, region)
    
    elif method == "proportion":
            volume_adjusted = proportion(ICV, region)
            
    elif method == "power_proportion":
        volume_adjusted = power_proportion(ICV, region)
            

        
    return(volume_adjusted)





